In [1]:
import urllib.request    # needed for download of the example database
import shutil            # needed for unziping of the example database

from sqlalchemy import * # package providing unified access to various databases
from sqlalchemy.orm import *
import os                # for checking existence/removing of a file
import pandas as pd

In [2]:
engine = create_engine("sqlite:///create.db",echo=False)
session = Session(engine)

%load_ext sql
%sql sqlite:///create.db

In [3]:
#return a dataframe of student summary table
def studeSumDF():
    stu_sum = pd.read_sql("""
SELECT Universityid, Name, Email, Evaluation_finished AS Evalution, SUM(Value) AS Total_Score
FROM Students
INNER JOIN Assignment USING(Universityid)
INNER JOIN Task USING (Taskid)
INNER JOIN Submission USING (Assignmentid)
INNER JOIN Score USING (Submissionid)
GROUP BY Universityid
""", engine)
    stu_sum.loc[stu_sum["Evalution"] != 0, "Evalution"] = "Finished"
    stu_sum.loc[stu_sum["Evalution"] == 0, "Evalution"] = "Pending"
    
    return(stu_sum)

#return a row of a single student summary table
def singleStudentSumDF(Universityid):
    stu_sum = studentsSumDF()
    stu_sing = stu_sum[stu_sum['UniversityID'] == Universityid]
    
    return(stu_sing)

studeSumDF()

,Universityid,Name,Email,Evalution,Total_Score
0,s3002428,Hanna Beck,s3002428@leidenuniv.nl,Pending,46.0
1,s3003252,Ellie Porter,s3003252@leidenuniv.nl,Pending,40.1
2,s3003316,Reiter Page,s3003316@leidenuniv.nl,Pending,35.7
3,s3004195,Cora Montgomery,s3004195@leidenuniv.nl,Pending,25.8
4,s3006578,Mary Waters,s3006578@leidenuniv.nl,Pending,44.8
...,...,...,...,...,...
85,s3095070,Betty Cummings,s3095070@leidenuniv.nl,Pending,29.1
86,s3095536,Felix Williamson,s3095536@leidenuniv.nl,Pending,49.5
87,s3095643,Lena Jensen,s3095643@leidenuniv.nl,Pending,20.5
88,s3096676,Thomas Frank,s3096676@leidenuniv.nl,Pending,51.6
